In [ ]:
#!pip install more-itertools

In [2]:
import pandas as pd
import numpy as np
import warnings
import random
import matplotlib.pyplot as plt
from more_itertools import sort_together

In [6]:
acci_df = pd.read_excel('/content/acci_info.xlsx')
person = list(acci_df['person_per_center'])
accident = list(acci_df['acci_per_center'])
print(len(person))
print(len(accident))

59
59


In [7]:
#초기 해 생성
def generate(num):
    e_list = []
    a = []
    b = []

    for i in range(num):
      e_chromosome = []
      for j in accident:
        chromsome = random.randrange(j+10, j + 20)
        e_chromosome.append(chromsome)
      e_list.append(e_chromosome)

      if sum(e_chromosome) <= 2060:
          a.append(e_chromosome)
      else:
          b.append(e_chromosome)

    return a

In [8]:
#적합도 함수 계산
def fitness(fm1, max_head):
  result = [0] * len(fm1)
  score = [0] * len(fm1)
  v_list = []
  for i in range(len(fm1)):
    v_list = []
    v = accident[0] / generation[i][0] #센터별 사고수/세대별 센터별 인원
    v_list.append(v)

    if sum(fm1[i]) < max_head:
      mean = np.mean(v_list) #표준편차(센터별 인원 합계/안전센터 수)
      result[i] += mean
    score[i] += 10.0 - result[i]

  return score

In [9]:
#교차 연산
def crossover(fm2):
    new_generation = []

    for i in range(50): # 1점 교차 방식
        parent = random.sample(fm2, 2)
        child1 = parent[0][0:11] + parent[1][11:23] #자손 염색체 2개씩 생성
        child2 = parent[1][0:11] + parent[0][11:23]
        random.shuffle(child1)
        random.shuffle(child2)
        new_generation.append(child1)
        new_generation.append(child2)

    return new_generation

In [10]:
#변이 연산
def mutate(fm3, prob):
    mutate_list = [0] * len(fm3)
    mutate_list = fm3

    for i in range(len(fm3)):
        if random.random() * 100 <= prob:
            mutate_list[i].pop()
            mutate_list[i].insert(random.randint(0,22).random.randint(10,200))

    return mutate_list

In [11]:
storage = []
storage_score = []
goal = 9.5
generation = generate(300) #generate(세대 수)
count = 300 #세대 수
x = 0

In [12]:
while x < count: #반복 시작
    x += 1
    fitness_score = fitness(generation, 2060) #적합도 평가: 총 인원 수 이하의 해만 점수 계산

    # 상위 20개의 해만 생존
    generation = sort_together([fitness_score, generation])[1]
    generation = generation[0:20]
    fitness_score = sorted(fitness_score, reverse = True)
    fitness_score = fitness_score[0:20]

    if fitness_score[0] >= 0.5:
        storage.append(generation[0])
        storage_score.append(fitness_score[0])

    storage = [i for _, i in sorted(zip(storage_score, storage), reverse = True)]
    storage_score = sorted(storage_score, reverse=True)

    # 조건 만족 시 반복 중단
    if fitness_score[0] >= goal:
        break

# crossover
generation = crossover(generation)

# mutation
generation = mutate(generation, 0.001)

# 최적해 산출
if fitness_score[0] >= goal: #count 안에 목표점수를 넘었으면 해당해가 최적해
    optimal_solution = generation[0]
else: #count안으로 목표점수를 못 넘었다면 모든 세대 중 가장 점수 높은 해가 최적해
    optimal_solution = storage[0]

print('지점별 재배치 결과: ', optimal_solution)
print('재배치 인원 합계: ', sum(optimal_solution))
print('총 세대수: ', x)

지점별 재배치 결과:  [72, 66, 66, 63, 57, 51, 43, 45, 43, 46, 42, 40, 37, 34, 39, 33, 33, 34, 34, 33, 33, 38, 39, 34, 37, 36, 35, 28, 33, 33, 34, 27, 28, 31, 27, 33, 33, 27, 28, 26, 25, 27, 22, 28, 21, 27, 24, 26, 22, 25, 19, 21, 25, 18, 17, 20, 19, 16, 21]
재배치 인원 합계:  1974
총 세대수:  300
